In [1]:
import uproot
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import math
from scipy.fft import fft, fftfreq
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import os
from plotly.subplots import make_subplots
from plotly import tools
import plotly.offline as pyo
import sys
from tqdm import tqdm

In [2]:
event_number = 606
files =uproot.open(f"/Users/danielcarber/Documents/SBND/Noise Analysis/data/waveform_output_{event_number}.root")
files['tpc_noise;1'].keys()

['UB_plane', 'VB_plane', 'YB_plane', 'UA_plane', 'VA_plane', 'YA_plane']

In [3]:
raw_rms = files['tpc_noise;1']['UB_plane'].array().to_list()
print(len(raw_rms))


6775360


In [ ]:
seaborn.set(rc={
    'figure.figsize':(20, 6),
})
seaborn.set_context('talk')
#plt.plot(range(len(ADC_array[0][0])),ADC_array[0][4231])
#plt.hist(signal_diff_coll,range = [0,300],bins = 100, label = "Coll")
#plt.hist(signal_diff_ind_1,range = [0,300],bins = 100,alpha = .6, label = "Ind 1")
#plt.hist(signal_diff_ind_2,range = [0,300],bins = 100, alpha = .2,label = "Ind 2")
#plt.scatter(df['channel_id'],df['raw_rms'],alpha = 1, label = "RMS before shift",s = 10)
#plt.scatter(df_2['channel_id'],df_2['raw_rms'],alpha = 1, label = "RMS after shift",s = 10)
plt.scatter(range(len(raw_rms)),raw_rms ,alpha = .8,s = 5, label = "RMS of Channels w/ signals")

#plt.hist(df['Time_diff'], range=[0, 2000],bins=50,alpha =1, label = "Time difference")
#plt.hist(selection['reco_dedx'][ep_mask_e], range=[0, 10], alpha = .4,bins=50,label = "Reco Electrons")
#plt.ylim(0,17.5)
plt.xlim(0,3415)
#plt.yscale("log")
#plt.legend(loc = "upper right")
plt.xlabel("Channels")
plt.ylabel("RMS")
plt.title("RMS of Run 10775")
plt.show()

In [4]:

Waveform_df = {}
ch_id = 0
raw_rms = files['tpc_noise;1']['VB_plane'].array().to_list()
for r,rms in tqdm(enumerate(raw_rms)):
    if r%3415 == 0:
        Waveform_df[f'Channel_{ch_id}'] = []
        Waveform_df[f'Channel_{ch_id}'].append(rms)
        ch_id +=1
    else:
        Waveform_df[f'Channel_{ch_id-1}'].append(rms)
        



        
Waveform_df = pd.DataFrame(Waveform_df)
waveform_max = []
for keys in Waveform_df:
    waveform_max.append(np.max(Waveform_df[keys]))
print(ch_id)

6775360it [00:01, 4367058.22it/s]


1984


In [5]:
print(np.size(Waveform_df))

6775360


In [6]:
wire_plane_list = ['UB','VB','YB','UA','VA','YA']
wire_df = {'Channel_id':[],'cryo':[],'tpc':[],'tpc':[],'plane':[],'rel_wire':[],'x_0':[],'y_0':[],'z_0':[],'x_1':[],'y_1':[],'z_1':[],'r':[]}
wire_txt = '/Users/danielcarber/Documents/SBND/Noise Analysis/sbn_noise_repo/sbnd/datafiles/Wire_lengths.txt'

with open(wire_txt) as f:
    for line in f:
        #print(line)
        currentline = line.split(" ")
        #print(currentline)
        wire_df['Channel_id'].append(int(currentline[0]))
        wire_df['cryo'].append(int(currentline[1]))
        wire_df['tpc'].append(int(currentline[2]))
        wire_df['plane'].append(int(currentline[3]))
        wire_df['rel_wire'].append(int(currentline[4]))
        wire_df['x_0'].append(float(currentline[5]))
        wire_df['y_0'].append(float(currentline[6]))
        wire_df['z_0'].append(float(currentline[7]))
        wire_df['x_1'].append(float(currentline[8]))
        wire_df['y_1'].append(float(currentline[9]))
        z_1 = currentline[10]
        #print(z_1[:-2])
        wire_df['z_1'].append(float(currentline[10][:-2]))
        length = np.sqrt(np.square(float(currentline[8])-float(currentline[5]))+np.square(float(currentline[9])-float(currentline[6]))+np.square(float(currentline[10][:-2])-float(currentline[7])))
        wire_df['r'].append(length)
#wire_df = pd.DataFrame(wire_df) 
ch_mask = wire_df['Channel_id'] == 1
print(wire_df['Channel_id'][1])
print((wire_df['y_0'][10]))



1
195.808


In [9]:
def RMS_calc(waveform_df):
    RMS = []
    for keys in waveform_df:
        print(keys)
        waveform  = waveform_df[keys]
        square = 0
        for tick in waveform:
            square += (tick*tick) - np.mean(waveform)
        mean = square / len(waveform)
        RMS.append(np.sqrt(mean))
    return RMS
def Max_calc(waveform_df):
    waveform_max = []
    for keys in Waveform_df:
        waveform_max.append(np.max(Waveform_df[keys]))
    return waveform_max
def Min_calc(waveform_df):
    waveform_min = []
    for keys in Waveform_df:
        waveform_min.append(np.min(Waveform_df[keys]))
    return waveform_min
def Range_calc(waveform_max):
    waveform_range = []
    for keys in Waveform_df:
        waveform_range.append(np.max(Waveform_df[keys])-np.min(Waveform_df[keys]))
    return waveform_range

In [10]:
Waveform_df = {}
ch_id = 0
raw_rms = files['tpc_noise;1']['UB_plane'].array().to_list()
for r,rms in tqdm(enumerate(raw_rms)):
    if r%3415 == 0:
        Waveform_df[f'Channel_{ch_id}'] = []
        Waveform_df[f'Channel_{ch_id}'].append(rms)
        ch_id +=1
    else:
        Waveform_df[f'Channel_{ch_id-1}'].append(rms)
        



        
Waveform_df = pd.DataFrame(Waveform_df)
waveform = RMS_calc(Waveform_df)

print(max(waveform),min(waveform),np.mean(waveform))
print(ch_id)
#ch_id = 10
color = []
x = []
y=[]
c = []
#fig = make_subplots(rows=1,cols=1)
for ch in tqdm(range(0,ch_id,1)):
    a = (wire_df['y_1'][ch]-wire_df['y_0'][ch])/(wire_df['z_1'][ch]-wire_df['z_0'][ch])
    b = wire_df['y_1'][ch] -a*wire_df['z_1'][ch]
    
    color.append(ch)
    for i in range(int(wire_df['r'][ch])):
        x_i = i*((wire_df['z_1'][ch]-wire_df['z_0'][ch])/int(wire_df['r'][ch]))+wire_df['z_0'][ch]
        x.append(i*((wire_df['z_1'][ch]-wire_df['z_0'][ch])/int(wire_df['r'][ch]))+wire_df['z_0'][ch])
        c.append(waveform[ch])
        y.append(a*x_i+b)
df = {'X [cm]':np.array(x),'Y [cm]':np.array(y),"ADC":np.array(c)}
#df = pd.DataFrame(df)
fig=px.scatter(df,x="X [cm]",y ="Y [cm]",color = "ADC",range_color=(0,4000),title="East TPC First Ind Wire Min Signal")

#fig.update_layout(xaxis2 = dict(range = [0,median+5]))
#fig.update_layout(margin = dict(r=200))
#fig.add_annotation(dict(font = dict(size = 10),xshift= 180,yshift=120,text = f"Mean RMS:{mean:.2f}",showarrow = False),row =1,col=2)
#fig.add_annotation(dict(font = dict(size = 10),xshift= 180,yshift=110,text = f"Median RMS:{median:.2f}",showarrow = False),row =1,col=2)


#fig.update_yaxes(title_text = "RMS [ADC]",row = 1, col = 1)

#fig.update_xaxes(title_text = "Wire length [cm]",row = 1, col = 1)

#fig.update_layout(xaxis2 = dict(range = [0,5]),xaxis4 = dict(range = [0,5]),xaxis6 = dict(range = [0,5]))
#fig.update_layout(yaxis = dict(range = [0,5]),yaxis3 = dict(range = [0,5]),yaxis5 = dict(range = [0,5]))
#fig.update_layout(xaxis = dict(range= [-10,700],tickmode = 'linear',dtick = 100))
fig.update_layout(height = 800, width = 1200,showlegend = False)

fig.write_image(f'/Users/danielcarber/Documents/SBND/Noise Analysis/Plots/Event_diagnostics/UB_plane_diagram_event_{event_number}.png')
print("Done")
#fig.show()

6775360it [00:01, 4217996.36it/s]


Channel_0
Channel_1
Channel_2
Channel_3
Channel_4
Channel_5
Channel_6
Channel_7
Channel_8
Channel_9
Channel_10
Channel_11
Channel_12
Channel_13
Channel_14
Channel_15
Channel_16
Channel_17
Channel_18
Channel_19
Channel_20
Channel_21
Channel_22
Channel_23
Channel_24
Channel_25
Channel_26
Channel_27
Channel_28
Channel_29
Channel_30
Channel_31
Channel_32
Channel_33
Channel_34
Channel_35
Channel_36
Channel_37
Channel_38
Channel_39
Channel_40
Channel_41
Channel_42
Channel_43
Channel_44
Channel_45
Channel_46
Channel_47
Channel_48
Channel_49
Channel_50
Channel_51
Channel_52
Channel_53
Channel_54
Channel_55
Channel_56
Channel_57
Channel_58
Channel_59
Channel_60
Channel_61
Channel_62
Channel_63
Channel_64
Channel_65
Channel_66
Channel_67
Channel_68
Channel_69
Channel_70
Channel_71
Channel_72
Channel_73
Channel_74
Channel_75
Channel_76
Channel_77
Channel_78
Channel_79
Channel_80
Channel_81
Channel_82
Channel_83
Channel_84
Channel_85
Channel_86
Channel_87
Channel_88
Channel_89
Channel_90
Channel_9

Channel_694
Channel_695
Channel_696
Channel_697
Channel_698
Channel_699
Channel_700
Channel_701
Channel_702
Channel_703
Channel_704
Channel_705
Channel_706
Channel_707
Channel_708
Channel_709
Channel_710
Channel_711
Channel_712
Channel_713
Channel_714
Channel_715
Channel_716
Channel_717
Channel_718
Channel_719
Channel_720
Channel_721
Channel_722
Channel_723
Channel_724
Channel_725
Channel_726
Channel_727
Channel_728
Channel_729
Channel_730
Channel_731
Channel_732
Channel_733
Channel_734
Channel_735
Channel_736
Channel_737
Channel_738
Channel_739
Channel_740
Channel_741
Channel_742
Channel_743
Channel_744
Channel_745
Channel_746
Channel_747
Channel_748
Channel_749
Channel_750
Channel_751
Channel_752
Channel_753
Channel_754
Channel_755
Channel_756
Channel_757
Channel_758
Channel_759
Channel_760
Channel_761
Channel_762
Channel_763
Channel_764
Channel_765
Channel_766
Channel_767
Channel_768
Channel_769
Channel_770
Channel_771
Channel_772
Channel_773
Channel_774
Channel_775
Channel_776
Chan

Channel_1350
Channel_1351
Channel_1352
Channel_1353
Channel_1354
Channel_1355
Channel_1356
Channel_1357
Channel_1358
Channel_1359
Channel_1360
Channel_1361
Channel_1362
Channel_1363
Channel_1364
Channel_1365
Channel_1366
Channel_1367
Channel_1368
Channel_1369
Channel_1370
Channel_1371
Channel_1372
Channel_1373
Channel_1374
Channel_1375
Channel_1376
Channel_1377
Channel_1378
Channel_1379
Channel_1380
Channel_1381
Channel_1382
Channel_1383
Channel_1384
Channel_1385
Channel_1386
Channel_1387
Channel_1388
Channel_1389
Channel_1390
Channel_1391
Channel_1392
Channel_1393
Channel_1394
Channel_1395
Channel_1396
Channel_1397
Channel_1398
Channel_1399
Channel_1400
Channel_1401
Channel_1402
Channel_1403
Channel_1404
Channel_1405
Channel_1406
Channel_1407
Channel_1408
Channel_1409
Channel_1410
Channel_1411
Channel_1412
Channel_1413
Channel_1414
Channel_1415
Channel_1416
Channel_1417
Channel_1418
Channel_1419
Channel_1420
Channel_1421
Channel_1422
Channel_1423
Channel_1424
Channel_1425
Channel_1426

Channel_1981
Channel_1982
Channel_1983
2090.061705044438 1994.678690902679 2040.984933674668
1984


100%|███████████████████████████████████████████████████████████████████| 1984/1984 [00:00<00:00, 5611.11it/s]


Done


In [ ]:
Waveform_df = {}
ch_id = 1984
raw_rms = files['tpc_noise;1']['VB_plane'].array().to_list()
for r,rms in tqdm(enumerate(raw_rms)):
    if r%3415 == 0:
        Waveform_df[f'Channel_{ch_id}'] = []
        Waveform_df[f'Channel_{ch_id}'].append(rms)
        ch_id +=1
    else:
        Waveform_df[f'Channel_{ch_id-1}'].append(rms)
        



        
Waveform_df = pd.DataFrame(Waveform_df)
waveform = RMS_calc(Waveform_df)

print(max(waveform),min(waveform),np.mean(waveform))
print(ch_id)
#ch_id = 10
color = []
x = []
y=[]
c = []
#fig = make_subplots(rows=1,cols=1)
for ch in tqdm(range(1984,ch_id,1)):
    a = (wire_df['y_1'][ch]-wire_df['y_0'][ch])/(wire_df['z_1'][ch]-wire_df['z_0'][ch])
    b = wire_df['y_1'][ch] -a*wire_df['z_1'][ch]
    
    color.append(ch)
    for i in range(int(wire_df['r'][ch])):
        x_i = i*((wire_df['z_1'][ch]-wire_df['z_0'][ch])/int(wire_df['r'][ch]))+wire_df['z_0'][ch]
        x.append(i*((wire_df['z_1'][ch]-wire_df['z_0'][ch])/int(wire_df['r'][ch]))+wire_df['z_0'][ch])
        #print(ch)
        c.append(waveform[ch-1984])
        y.append(a*x_i+b)
df = {'X [cm]':np.array(x),'Y [cm]':np.array(y),"ADC":np.array(c)}
#df = pd.DataFrame(df)
fig=px.scatter(df,x="X [cm]",y ="Y [cm]",color = "ADC",range_color=(0,4000),title="East TPC Second Ind Wire Min Signal")

#fig.update_layout(xaxis2 = dict(range = [0,median+5]))
#fig.update_layout(margin = dict(r=200))
#fig.add_annotation(dict(font = dict(size = 10),xshift= 180,yshift=120,text = f"Mean RMS:{mean:.2f}",showarrow = False),row =1,col=2)
#fig.add_annotation(dict(font = dict(size = 10),xshift= 180,yshift=110,text = f"Median RMS:{median:.2f}",showarrow = False),row =1,col=2)


#fig.update_yaxes(title_text = "RMS [ADC]",row = 1, col = 1)

#fig.update_xaxes(title_text = "Wire length [cm]",row = 1, col = 1)

#fig.update_layout(xaxis2 = dict(range = [0,5]),xaxis4 = dict(range = [0,5]),xaxis6 = dict(range = [0,5]))
#fig.update_layout(yaxis = dict(range = [0,5]),yaxis3 = dict(range = [0,5]),yaxis5 = dict(range = [0,5]))
#fig.update_layout(xaxis = dict(range= [-10,700],tickmode = 'linear',dtick = 100))
fig.update_layout(height = 800, width = 1200,showlegend = False)

fig.write_image(f'/Users/danielcarber/Documents/SBND/Noise Analysis/Plots/Event_diagnostics/VB_plane_diagram_{event_number}.png')
print("Done")
#fig.show()

In [ ]:
Waveform_df = {}
ch_id = 3968
raw_rms = files['tpc_noise;1']['YB_plane'].array().to_list()
for r,rms in tqdm(enumerate(raw_rms)):
    if r%3415 == 0:
        Waveform_df[f'Channel_{ch_id}'] = []
        Waveform_df[f'Channel_{ch_id}'].append(rms)
        ch_id +=1
    else:
        Waveform_df[f'Channel_{ch_id-1}'].append(rms)
        



        
Waveform_df = pd.DataFrame(Waveform_df)
waveform = RMS_calc(Waveform_df)

print(max(waveform),min(waveform),np.mean(waveform))
print(ch_id)
#ch_id = 10
color = []
x = []
y=[]
c = []
#fig = make_subplots(rows=1,cols=1)
for ch in tqdm(range(3968,ch_id,1)):
    #a = (wire_df['y_1'][ch]-wire_df['y_0'][ch])/(wire_df['z_1'][ch]-wire_df['z_0'][ch])
    #b = wire_df['y_1'][ch] -a*wire_df['z_1'][ch]
    
    color.append(ch)
    for i in range(int(wire_df['y_0'][ch]),int(wire_df['y_1'][ch]),1):
        x_i = wire_df['z_0'][ch]
        x.append(wire_df['z_0'][ch])
        #print(ch)
        c.append(waveform[ch-3968])
        y.append(i)
df = {'X [cm]':np.array(x),'Y [cm]':np.array(y),"ADC":np.array(c)}
#df = pd.DataFrame(df)
fig=px.scatter(df,x="X [cm]",y ="Y [cm]",color = "ADC",range_color=(0,1000),title="East TPC Coll Wire Min Signal")

#fig.update_layout(xaxis2 = dict(range = [0,median+5]))
#fig.update_layout(margin = dict(r=200))
#fig.add_annotation(dict(font = dict(size = 10),xshift= 180,yshift=120,text = f"Mean RMS:{mean:.2f}",showarrow = False),row =1,col=2)
#fig.add_annotation(dict(font = dict(size = 10),xshift= 180,yshift=110,text = f"Median RMS:{median:.2f}",showarrow = False),row =1,col=2)


#fig.update_yaxes(title_text = "RMS [ADC]",row = 1, col = 1)

#fig.update_xaxes(title_text = "Wire length [cm]",row = 1, col = 1)

#fig.update_layout(xaxis2 = dict(range = [0,5]),xaxis4 = dict(range = [0,5]),xaxis6 = dict(range = [0,5]))
#fig.update_layout(yaxis = dict(range = [0,5]),yaxis3 = dict(range = [0,5]),yaxis5 = dict(range = [0,5]))
#fig.update_layout(xaxis = dict(range= [-10,700],tickmode = 'linear',dtick = 100))
fig.update_layout(height = 800, width = 1200,showlegend = False)

fig.write_image(f'/Users/danielcarber/Documents/SBND/Noise Analysis/Plots/Event_diagnostics/YB_plane_diagram_{event_number}.png')
print("Done")
#fig.show()

In [ ]:
Waveform_df = {}
ch_id = 5632
raw_rms = files['tpc_noise;1']['UA_plane'].array().to_list()
for r,rms in tqdm(enumerate(raw_rms)):
    if r%3415 == 0:
        Waveform_df[f'Channel_{ch_id}'] = []
        Waveform_df[f'Channel_{ch_id}'].append(rms)
        ch_id +=1
    else:
        Waveform_df[f'Channel_{ch_id-1}'].append(rms)
        



        
Waveform_df = pd.DataFrame(Waveform_df)
waveform = RMS_calc(Waveform_df)

print(max(waveform),min(waveform),np.mean(waveform))
print(ch_id)
#ch_id = 10
color = []
x = []
y=[]
c = []
#fig = make_subplots(rows=1,cols=1)
for ch in tqdm(range(5632,ch_id,1)):
    a = (wire_df['y_1'][ch]-wire_df['y_0'][ch])/(wire_df['z_1'][ch]-wire_df['z_0'][ch])
    b = wire_df['y_1'][ch] -a*wire_df['z_1'][ch]
    
    color.append(ch)
    for i in range(int(wire_df['r'][ch])):
        x_i = i*((wire_df['z_1'][ch]-wire_df['z_0'][ch])/int(wire_df['r'][ch]))+wire_df['z_0'][ch]
        x.append(i*((wire_df['z_1'][ch]-wire_df['z_0'][ch])/int(wire_df['r'][ch]))+wire_df['z_0'][ch])
        #print(ch)
        c.append(waveform[ch-5632])
        y.append(a*x_i+b)
df = {'X [cm]':np.array(x),'Y [cm]':np.array(y),"ADC":np.array(c)}
#df = pd.DataFrame(df)
fig=px.scatter(df,x="X [cm]",y ="Y [cm]",color = "ADC",range_color=(1700,2800),title="West TPC First Ind Wire Min Signal")

#fig.update_layout(xaxis2 = dict(range = [0,median+5]))
#fig.update_layout(margin = dict(r=200))
#fig.add_annotation(dict(font = dict(size = 10),xshift= 180,yshift=120,text = f"Mean RMS:{mean:.2f}",showarrow = False),row =1,col=2)
#fig.add_annotation(dict(font = dict(size = 10),xshift= 180,yshift=110,text = f"Median RMS:{median:.2f}",showarrow = False),row =1,col=2)


#fig.update_yaxes(title_text = "RMS [ADC]",row = 1, col = 1)

#fig.update_xaxes(title_text = "Wire length [cm]",row = 1, col = 1)

#fig.update_layout(xaxis2 = dict(range = [0,5]),xaxis4 = dict(range = [0,5]),xaxis6 = dict(range = [0,5]))
#fig.update_layout(yaxis = dict(range = [0,5]),yaxis3 = dict(range = [0,5]),yaxis5 = dict(range = [0,5]))
#fig.update_layout(xaxis = dict(range= [-10,700],tickmode = 'linear',dtick = 100))
fig.update_layout(height = 800, width = 1200,showlegend = False)

fig.write_image(f'/Users/danielcarber/Documents/SBND/Noise Analysis/Plots/Event_diagnostics/UA_plane_diagram_{event_number}.png')
print("Done")
#fig.show()

In [ ]:
Waveform_df = {}
ch_id = 7616
raw_rms = files['tpc_noise;1']['VA_plane'].array().to_list()
for r,rms in tqdm(enumerate(raw_rms)):
    if r%3415 == 0:
        Waveform_df[f'Channel_{ch_id}'] = []
        Waveform_df[f'Channel_{ch_id}'].append(rms)
        ch_id +=1
    else:
        Waveform_df[f'Channel_{ch_id-1}'].append(rms)
        



        
Waveform_df = pd.DataFrame(Waveform_df)
waveform = RMS_calc(Waveform_df)
print(max(waveform),min(waveform),np.mean(waveform))
print(ch_id)
#ch_id = 10
color = []
x = []
y=[]
c = []
#fig = make_subplots(rows=1,cols=1)
for ch in tqdm(range(7616,ch_id,1)):
    a = (wire_df['y_1'][ch]-wire_df['y_0'][ch])/(wire_df['z_1'][ch]-wire_df['z_0'][ch])
    b = wire_df['y_1'][ch] -a*wire_df['z_1'][ch]
    
    color.append(ch)
    for i in range(int(wire_df['r'][ch])):
        x_i = i*((wire_df['z_1'][ch]-wire_df['z_0'][ch])/int(wire_df['r'][ch]))+wire_df['z_0'][ch]
        x.append(i*((wire_df['z_1'][ch]-wire_df['z_0'][ch])/int(wire_df['r'][ch]))+wire_df['z_0'][ch])
        #print(ch)
        c.append(waveform[ch-7616])
        y.append(a*x_i+b)
df = {'X [cm]':np.array(x),'Y [cm]':np.array(y),"ADC":np.array(c)}
#df = pd.DataFrame(df)
fig=px.scatter(df,x="X [cm]",y ="Y [cm]",color = "ADC",range_color=(1700,2800),title="West TPC Second Ind Wire Min Signal")

#fig.update_layout(xaxis2 = dict(range = [0,median+5]))
#fig.update_layout(margin = dict(r=200))
#fig.add_annotation(dict(font = dict(size = 10),xshift= 180,yshift=120,text = f"Mean RMS:{mean:.2f}",showarrow = False),row =1,col=2)
#fig.add_annotation(dict(font = dict(size = 10),xshift= 180,yshift=110,text = f"Median RMS:{median:.2f}",showarrow = False),row =1,col=2)


#fig.update_yaxes(title_text = "RMS [ADC]",row = 1, col = 1)

#fig.update_xaxes(title_text = "Wire length [cm]",row = 1, col = 1)

#fig.update_layout(xaxis2 = dict(range = [0,5]),xaxis4 = dict(range = [0,5]),xaxis6 = dict(range = [0,5]))
#fig.update_layout(yaxis = dict(range = [0,5]),yaxis3 = dict(range = [0,5]),yaxis5 = dict(range = [0,5]))
#fig.update_layout(xaxis = dict(range= [-10,700],tickmode = 'linear',dtick = 100))
fig.update_layout(height = 800, width = 1200,showlegend = False)

fig.write_image(f'/Users/danielcarber/Documents/SBND/Noise Analysis/Plots/Event_diagnostics/VA_plane_diagram_{event_number}.png')
print("Done")
#fig.show()

In [ ]:
Waveform_df = {}
ch_id = 9600
raw_rms = files['tpc_noise;1']['YA_plane'].array().to_list()
for r,rms in tqdm(enumerate(raw_rms)):
    if r%3415 == 0:
        Waveform_df[f'Channel_{ch_id}'] = []
        Waveform_df[f'Channel_{ch_id}'].append(rms)
        ch_id +=1
    else:
        Waveform_df[f'Channel_{ch_id-1}'].append(rms)
        



        
Waveform_df = pd.DataFrame(Waveform_df)
waveform = RMS_calc(Waveform_df)

print(ch_id)
#ch_id = 10
color = []
x = []
y=[]
c = []
#fig = make_subplots(rows=1,cols=1)
for ch in tqdm(range(9600,ch_id,1)):
    a = (wire_df['y_1'][ch]-wire_df['y_0'][ch])/(wire_df['z_1'][ch]-wire_df['z_0'][ch])
    b = wire_df['y_1'][ch] -a*wire_df['z_1'][ch]
    
    color.append(ch)
    for i in range(int(wire_df['r'][ch])):
        x_i = i*((wire_df['z_1'][ch]-wire_df['z_0'][ch])/int(wire_df['r'][ch]))+wire_df['z_0'][ch]
        x.append(i*((wire_df['z_1'][ch]-wire_df['z_0'][ch])/int(wire_df['r'][ch]))+wire_df['z_0'][ch])
        #print(ch)
        c.append(waveform[ch-9600])
        y.append(a*x_i+b)
df = {'X [cm]':np.array(x),'Y [cm]':np.array(y),"ADC":np.array(c)}
#df = pd.DataFrame(df)
fig=px.scatter(df,x="X [cm]",y ="Y [cm]",color = "ADC",range_color=(0,4000),title="West TPC Collection Wire Max Signal")

#fig.update_layout(xaxis2 = dict(range = [0,median+5]))
#fig.update_layout(margin = dict(r=200))
#fig.add_annotation(dict(font = dict(size = 10),xshift= 180,yshift=120,text = f"Mean RMS:{mean:.2f}",showarrow = False),row =1,col=2)
#fig.add_annotation(dict(font = dict(size = 10),xshift= 180,yshift=110,text = f"Median RMS:{median:.2f}",showarrow = False),row =1,col=2)


#fig.update_yaxes(title_text = "RMS [ADC]",row = 1, col = 1)

#fig.update_xaxes(title_text = "Wire length [cm]",row = 1, col = 1)

#fig.update_layout(xaxis2 = dict(range = [0,5]),xaxis4 = dict(range = [0,5]),xaxis6 = dict(range = [0,5]))
#fig.update_layout(yaxis = dict(range = [0,5]),yaxis3 = dict(range = [0,5]),yaxis5 = dict(range = [0,5]))
#fig.update_layout(xaxis = dict(range= [-10,700],tickmode = 'linear',dtick = 100))
fig.update_layout(height = 800, width = 1200,showlegend = False)

fig.write_image(f'/Users/danielcarber/Documents/SBND/Noise Analysis/Plots/Event_diagnostics/YA_plane_diagram_{event_number}.png')
print("Done")
#fig.show()